In [ ]:
import streamlit as st
import pickle
import json
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

# Page config
st.set_page_config(
    page_title="AI Career Chatbot",
    page_icon="🤖",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
    .stApp {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    }
    .chat-message {
        padding: 1.5rem;
        border-radius: 0.5rem;
        margin-bottom: 1rem;
        display: flex;
        flex-direction: column;
    }
    .user-message {
        background-color: #4facfe;
        color: white;
    }
    .bot-message {
        background-color: #f0f0f0;
        color: black;
    }
</style>
""", unsafe_allow_html=True)

# Load model and artifacts
@st.cache_resource
def load_model_artifacts():
    model = keras.models.load_model('/content/career_chatbot_model.h5')

    with open('/content/tokenizer.pickle', 'rb') as f:
        tokenizer = pickle.load(f)

    with open('/content/label_encoder.pickle', 'rb') as f:
        label_encoder = pickle.load(f)

    with open('/content/response_templates.json', 'r') as f:
        response_templates = json.load(f)

    return model, tokenizer, label_encoder, response_templates

model, tokenizer, label_encoder, response_templates = load_model_artifacts()

# Preprocessor class
class TextPreprocessor:
    def __init__(self):
        self.stopwords = set(['a', 'an', 'the', 'is', 'am', 'are'])

    def preprocess(self, text):
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s]', '', text)
        return ' '.join(text.split())

preprocessor = TextPreprocessor()

# Prediction function
def predict_and_respond(user_input):
    processed = preprocessor.preprocess(user_input)
    sequence = tokenizer.texts_to_sequences([processed])
    padded = pad_sequences(sequence, maxlen=20, padding='post')

    prediction = model.predict(padded, verbose=0)[0]
    intent_idx = np.argmax(prediction)
    confidence = float(prediction[intent_idx])
    intent = label_encoder.classes_[intent_idx]

    template = response_templates.get(intent, "I can help with career guidance!")
    response = template.format(confidence=f"{confidence*100:.1f}")

    return response, intent, confidence

# App title
st.title("🤖 AI Career Guidance Chatbot")
st.markdown("### NLP-Powered Career Advisor")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Hello! I'm your AI Career Guidance Assistant. How can I help you today?"}
    ]

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Ask me about your career..."):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # Get bot response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response, intent, confidence = predict_and_respond(prompt)
            st.markdown(response)
            st.caption(f"Intent: {intent} | Confidence: {confidence*100:.1f}%")

    # Add bot response to history
    st.session_state.messages.append({"role": "assistant", "content": response})

# Sidebar with quick actions
with st.sidebar:
    st.header("Quick Actions")

    if st.button("🎯 What skills should I learn?"):
        st.session_state.messages.append({"role": "user", "content": "What skills should I learn?"})
        st.rerun()

    if st.button("💼 Interview preparation"):
        st.session_state.messages.append({"role": "user", "content": "How do I prepare for interviews?"})
        st.rerun()

    if st.button("🚀 Career exploration"):
        st.session_state.messages.append({"role": "user", "content": "Tell me about career paths"})
        st.rerun()

    if st.button("💰 Salary information"):
        st.session_state.messages.append({"role": "user", "content": "Show me salary information"})
        st.rerun()

    st.divider()

    if st.button("🗑️ Clear Chat"):
        st.session_state.messages = [
            {"role": "assistant", "content": "Hello! I'm your AI Career Guidance Assistant. How can I help you today?"}
        ]
        st.rerun()

In [8]:
# Step 1 — Define and Save Streamlit App
code = """
# ⬇️ Paste your entire Streamlit chatbot code below this line

import streamlit as st
import pickle
import json
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

st.set_page_config(page_title="AI Career Chatbot", page_icon="🤖", layout="wide")

st.markdown('''
<style>
    .stApp {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    }
</style>
''', unsafe_allow_html=True)

@st.cache_resource
def load_model_artifacts():
    model = keras.models.load_model('career_chatbot_model.h5')
    with open('tokenizer.pickle', 'rb') as f:
        tokenizer = pickle.load(f)
    with open('label_encoder.pickle', 'rb') as f:
        label_encoder = pickle.load(f)
    with open('response_templates.json', 'r') as f:
        response_templates = json.load(f)
    return model, tokenizer, label_encoder, response_templates

model, tokenizer, label_encoder, response_templates = load_model_artifacts()

class TextPreprocessor:
    def preprocess(self, text):
        text = text.lower()
        text = re.sub(r'[^a-z0-9\\s]', '', text)
        return ' '.join(text.split())

preprocessor = TextPreprocessor()

def predict_and_respond(user_input):
    processed = preprocessor.preprocess(user_input)
    sequence = tokenizer.texts_to_sequences([processed])
    padded = pad_sequences(sequence, maxlen=20, padding='post')
    prediction = model.predict(padded, verbose=0)[0]
    intent_idx = np.argmax(prediction)
    confidence = float(prediction[intent_idx])
    intent = label_encoder.classes_[intent_idx]
    template = response_templates.get(intent, "I can help with career guidance!")
    response = template.format(confidence=f"{confidence*100:.1f}")
    return response, intent, confidence

st.title("🤖 AI Career Guidance Chatbot")
st.markdown("### NLP-Powered Career Advisor")

if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Hello! I'm your AI Career Guidance Assistant. How can I help you today?"}
    ]

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Ask me about your career..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response, intent, confidence = predict_and_respond(prompt)
            st.markdown(response)
            st.caption(f"Intent: {intent} | Confidence: {confidence*100:.1f}%")
    st.session_state.messages.append({"role": "assistant", "content": response})

with st.sidebar:
    st.header("Quick Actions")
    if st.button("🎯 What skills should I learn?"):
        st.session_state.messages.append({"role": "user", "content": "What skills should I learn?"})
        st.rerun()
    if st.button("💼 Interview preparation"):
        st.session_state.messages.append({"role": "user", "content": "How do I prepare for interviews?"})
        st.rerun()
    if st.button("🚀 Career exploration"):
        st.session_state.messages.append({"role": "user", "content": "Tell me about career paths"})
        st.rerun()
    if st.button("💰 Salary information"):
        st.session_state.messages.append({"role": "user", "content": "Show me salary information"})
        st.rerun()
    st.divider()
    if st.button("🗑️ Clear Chat"):
        st.session_state.messages = [
            {"role": "assistant", "content": "Hello! I'm your AI Career Guidance Assistant. How can I help you today?"}
        ]
        st.rerun()
"""

# Step 2 — Save to file
with open("career_chatbot_app.py", "w", encoding="utf-8") as f:
    f.write(code)

print("✅ career_chatbot_app.py saved successfully!")


✅ career_chatbot_app.py saved successfully!


In [ ]:
!npm install -g localtunnel
!streamlit run career_chatbot_app.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.16.139.146:8502

⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦your url is: https://bumpy-dodos-flow.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:12637 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙

In [9]:
with open("career_chatbot_app.py", "w", encoding="utf-8") as f:
    f.write(code)
print("✅ career_chatbot_app.py saved successfully!")

✅ career_chatbot_app.py saved successfully!


In [2]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.1 MB/s eta 0:00:00
